In [9]:
import locale
import pandas as pd
import numpy as np
from scipy import stats

def formatear_moneda(valor):
    """
    Formatea un valor numérico como una cadena de texto con formato monetario.

    :param valor: El valor numérico a formatear.
    :param locale_str: La configuración regional a usar para el formateo.
    :return: Una cadena de texto con el valor formateado como moneda.
    """
    locale.setlocale(locale.LC_ALL, '')
    return locale.currency(valor, grouping=True)


def compute_stats(df, column, alpha = 0.95, formatter=lambda x:x):
    column = df[column]

    mean = column.mean()
    min = column.min()
    max = column.max()
    std = column.std()

    n = len(column)
    stderr = std / np.sqrt(n)
    h = stderr * stats.t.ppf((1 + alpha) / 2, n - 1)
    bottom_top_interval = mean - h
    top_interval = mean + h

    return {
        "desviación estándar": formatter(std),
        "mínimo": formatter(min),
        "mínimo del intervalo de confianza": formatter(bottom_top_interval),
        "media": formatter(mean),
        "máximo del intervalo de confianza": formatter(top_interval),
        "máximo": formatter(max)       
    }


In [10]:
df = pd.read_csv('clients.csv')
df.head()

,Unnamed: 0,simulation,type,arrive_time,requirement,waiting_room_arrive_time,ticker,attention_start_time,price,leave_time,leakage_time,worker_helper
0,0,0,C,9.013689,1.0,9.017396,1.0,9.017396,85715.118162,9.172273,NaN,0.0
1,1,0,B,9.083221,3.0,9.087543,1.0,9.087543,0.000000,9.088568,NaN,1.0
2,2,0,A,9.085701,2.0,9.093203,1.0,9.093203,-71186.201167,9.139125,NaN,1.0
3,3,0,A,9.088390,3.0,9.097473,2.0,9.139125,0.000000,9.213533,NaN,1.0
4,4,0,A,9.092578,2.0,9.101248,3.0,9.101248,-95233.400440,9.171817,NaN,3.0


In [11]:
df['leakage'] = df['leave_time'].isna()

df['attention_time'] = np.where(
    ~df['attention_start_time'].isna(),
    df['leave_time'] - df['attention_start_time'], 
    df['attention_start_time']
)

df['system_time'] = np.where(
    ~df['leakage'], 
    df['leave_time'] - df['arrive_time'], 
    df['leakage_time'] - df['arrive_time']
)

df['waiting_time'] = np.where(
    ~df['waiting_room_arrive_time'].isna(), 
    df['system_time'] + df['arrive_time'] - df["waiting_room_arrive_time"], 
    df['waiting_room_arrive_time']
)

df.head()

,Unnamed: 0,simulation,type,arrive_time,requirement,waiting_room_arrive_time,ticker,attention_start_time,price,leave_time,leakage_time,worker_helper,leakage,attention_time,system_time,waiting_time
0,0,0,C,9.013689,1.0,9.017396,1.0,9.017396,85715.118162,9.172273,NaN,0.0,False,0.154877,0.158584,0.154877
1,1,0,B,9.083221,3.0,9.087543,1.0,9.087543,0.000000,9.088568,NaN,1.0,False,0.001025,0.005347,0.001025
2,2,0,A,9.085701,2.0,9.093203,1.0,9.093203,-71186.201167,9.139125,NaN,1.0,False,0.045922,0.053424,0.045922
3,3,0,A,9.088390,3.0,9.097473,2.0,9.139125,0.000000,9.213533,NaN,1.0,False,0.074408,0.125143,0.116060
4,4,0,A,9.092578,2.0,9.101248,3.0,9.101248,-95233.400440,9.171817,NaN,3.0,False,0.070569,0.079239,0.070569


In [12]:
group = df.groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total = {"total de clientes fugados": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados,3.764186,9.069767,22.668684,22.742469,22.816255,38.506876


In [13]:
group = df[df['type'] == "A"].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_A = {"total de clientes de tipo A fugados": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_A).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes de tipo A fugados,5.52034,8.62069,25.620963,25.729173,25.837383,47.017544


In [14]:
group = df[df['type'] == "B"].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_B = {"total de clientes de tipo B fugados": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_B).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes de tipo B fugados,4.024278,0.0,11.768232,11.847116,11.926,27.731092


In [15]:
group = df[df['type'] == "C"].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_C = {"total de clientes de tipo C fugados": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_C).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes de tipo C fugados,7.13955,7.079646,28.09868,28.238629,28.378579,70.434783


In [16]:
group = df[df['arrive_time'] < 12].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_morning = {"total de clientes fugados en la mañana": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_morning).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados en la mañana,4.847832,0.0,8.293371,8.388399,8.483426,31.491713


In [17]:
group = df[(df['arrive_time'] >= 12) & (df['arrive_time'] < 14)].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_noon = {"total de clientes fugados al medio día": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_noon).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados al medio día,7.325621,16.37931,45.304531,45.448127,45.591724,70.992366


In [18]:
group = df[(df['arrive_time'] >= 14)].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_tarde = {"total de clientes fugados en la tarde": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_tarde).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados en la tarde,5.326304,3.389831,18.275066,18.379472,18.483879,45.73991


In [19]:
group = df.groupby('simulation').agg({
    "price": "sum",
})

money = {"total de ingresos": compute_stats(group, 'price', formatter=formatear_moneda) }

pd.DataFrame(money).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de ingresos,"$2,979,262.77","-$5,597,420.86","$7,169,938.22","$7,228,337.77","$7,286,737.31","$16,960,315.23"


In [20]:
group = df[df['type'] == "A"].groupby('simulation').agg({
    "system_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['system_time'] / group['arrive_time'] 

system_time_A = {"Tiempo promedio en el sistema clientes tipo A": compute_stats(group, 'mean') }

pd.DataFrame(system_time_A).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en el sistema clientes tipo A,0.062718,0.178073,0.378683,0.379913,0.381142,0.656886


In [21]:
group = df[df['type'] == "B"].groupby('simulation').agg({
    "system_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['system_time'] / group['arrive_time'] 

system_time_B = {"Tiempo promedio en el sistema clientes tipo B": compute_stats(group, 'mean') }

pd.DataFrame(system_time_B).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en el sistema clientes tipo B,0.034898,0.154806,0.26022,0.260904,0.261588,0.441282


In [22]:
group = df[df['type'] == "C"].groupby('simulation').agg({
    "system_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['system_time'] / group['arrive_time'] 

system_time_C = {"Tiempo promedio en el sistema clientes tipo C": compute_stats(group, 'mean') }

pd.DataFrame(system_time_C).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en el sistema clientes tipo C,0.08781,0.171464,0.409781,0.411502,0.413224,0.944205


In [23]:
group = df[~df['waiting_time'].isna() & (df['type'] == "A")].groupby('simulation').agg({
    "waiting_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['waiting_time'] / group['arrive_time'] 

waiting_time_A = {"Tiempo promedio en la sala de espera clientes tipo A": compute_stats(group, 'mean') }

pd.DataFrame(waiting_time_A).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en la sala de espera clientes tipo A,0.061263,0.15878,0.349064,0.350265,0.351466,0.618759


In [24]:
group = df[~df['waiting_time'].isna() & (df['type'] == "B")].groupby('simulation').agg({
    "waiting_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['waiting_time'] / group['arrive_time'] 

waiting_time_B = {"Tiempo promedio en la sala de espera clientes tipo B": compute_stats(group, 'mean') }

pd.DataFrame(waiting_time_B).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en la sala de espera clientes tipo B,0.02368,0.133851,0.215209,0.215673,0.216137,0.325201


In [25]:
group = df[~df['waiting_time'].isna() & (df['type'] == "C")].groupby('simulation').agg({
    "waiting_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['waiting_time'] / group['arrive_time'] 

waiting_time_C = {"Tiempo promedio en la sala de espera clientes tipo C": compute_stats(group, 'mean') }

pd.DataFrame(waiting_time_C).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en la sala de espera clientes tipo C,0.093341,0.128139,0.379374,0.381204,0.383034,1.025052


In [26]:
group = df[(df['worker_helper'] == 0.0)].groupby('simulation').agg({
    "attention_time": "sum",
})

group['mean'] = group['attention_time'] / (18-9-1) * 100

worker_1_time = {"Porcentaje de tiempo de trabajo el modulo 1 empleado en la atención de clientes": compute_stats(group, 'mean') }

pd.DataFrame(worker_1_time).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Porcentaje de tiempo de trabajo el modulo 1 empleado en la atención de clientes,4.884373,69.094649,94.281461,94.377204,94.472948,111.481161


In [27]:
group = df[(df['worker_helper'] == 1.0)].groupby('simulation').agg({
    "attention_time": "sum",
})

group['mean'] = group['attention_time'] / (18-9-1) * 100

worker_2_time = {"Porcentaje de tiempo de trabajo el modulo 2 empleado en la atención de clientes": compute_stats(group, 'mean') }

pd.DataFrame(worker_2_time).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Porcentaje de tiempo de trabajo el modulo 2 empleado en la atención de clientes,3.653784,76.476184,97.750334,97.821956,97.893577,112.15373


In [28]:
group = df[(df['worker_helper'] == 2.0)].groupby('simulation').agg({
    "attention_time": "sum",
})

group['mean'] = group['attention_time'] / (18-9-1) * 100

worker_3_time = {"Porcentaje de tiempo de trabajo el modulo 3 empleado en la atención de clientes": compute_stats(group, 'mean') }

pd.DataFrame(worker_3_time).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Porcentaje de tiempo de trabajo el modulo 3 empleado en la atención de clientes,8.382979,49.117568,81.833822,81.998145,82.162469,107.376635


In [29]:
group = df[(df['worker_helper'] == 3.0)].groupby('simulation').agg({
    "attention_time": "sum",
})

group['mean'] = group['attention_time'] / (18-9-1) * 100

worker_4_time = {"Porcentaje de tiempo de trabajo el modulo 4 empleado en la atención de clientes": compute_stats(group, 'mean') }

pd.DataFrame(worker_4_time).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Porcentaje de tiempo de trabajo el modulo 4 empleado en la atención de clientes,6.864045,59.026616,90.162441,90.29699,90.431539,109.531499


# Resultados 

In [30]:
# Estadisticas de la fuga de los clientes

pd.DataFrame({
    **leakage_total,
    **leakage_total_A,
    **leakage_total_B,
    **leakage_total_C, 
    **leakage_total_morning,
    **leakage_total_noon,
    **leakage_total_tarde,
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados,3.764186,9.069767,22.668684,22.742469,22.816255,38.506876
total de clientes de tipo A fugados,5.520340,8.620690,25.620963,25.729173,25.837383,47.017544
total de clientes de tipo B fugados,4.024278,0.000000,11.768232,11.847116,11.926000,27.731092
total de clientes de tipo C fugados,7.139550,7.079646,28.098680,28.238629,28.378579,70.434783
total de clientes fugados en la mañana,4.847832,0.000000,8.293371,8.388399,8.483426,31.491713
total de clientes fugados al medio día,7.325621,16.379310,45.304531,45.448127,45.591724,70.992366
total de clientes fugados en la tarde,5.326304,3.389831,18.275066,18.379472,18.483879,45.739910


In [31]:
# Estadisticas de ingresos

pd.DataFrame(money).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de ingresos,"$2,979,262.77","-$5,597,420.86","$7,169,938.22","$7,228,337.77","$7,286,737.31","$16,960,315.23"


In [32]:
# Estadisticas de tiempo en el sistema

pd.DataFrame({
    **system_time_A,
    **system_time_B,
    **system_time_C,
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en el sistema clientes tipo A,0.062718,0.178073,0.378683,0.379913,0.381142,0.656886
Tiempo promedio en el sistema clientes tipo B,0.034898,0.154806,0.260220,0.260904,0.261588,0.441282
Tiempo promedio en el sistema clientes tipo C,0.087810,0.171464,0.409781,0.411502,0.413224,0.944205


In [33]:
# Estadisticas de tiempo en la lista de espera

pd.DataFrame({
    **waiting_time_A,
    **waiting_time_B,
    **waiting_time_C,
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en la sala de espera clientes tipo A,0.061263,0.158780,0.349064,0.350265,0.351466,0.618759
Tiempo promedio en la sala de espera clientes tipo B,0.023680,0.133851,0.215209,0.215673,0.216137,0.325201
Tiempo promedio en la sala de espera clientes tipo C,0.093341,0.128139,0.379374,0.381204,0.383034,1.025052


In [34]:
# Estadisticas de los trabajadores

group = df.groupby('simulation').agg({
    'worker_helper': [
        ('0', lambda x: (x == 0).sum()/ x.count() * 100), 
        ('1', lambda x: (x == 1).sum()/ x.count() * 100), 
        ('2', lambda x: (x == 2).sum()/ x.count() * 100), 
        ('3', lambda x: (x == 3).sum()/ x.count() * 100)  
    ],
})


group.columns = ['_'.join(col).strip() for col in group.columns.values]

workers = {
    "Porciento de clientes atendidos por el modulo 1": compute_stats(group, 'worker_helper_0'), 
    "Porciento de clientes atendidos por el modulo 2": compute_stats(group, 'worker_helper_1'), 
    "Porciento de clientes atendidos por el modulo 3": compute_stats(group, 'worker_helper_2'), 
    "Porciento de clientes atendidos por el modulo 4": compute_stats(group, 'worker_helper_3'), 
}

pd.DataFrame(workers).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Porciento de clientes atendidos por el modulo 1,2.020245,16.793893,24.046699,24.086300,24.125901,33.512064
Porciento de clientes atendidos por el modulo 2,2.330251,19.417476,28.581924,28.627602,28.673279,37.215190
Porciento de clientes atendidos por el modulo 3,2.146677,12.335958,20.857695,20.899775,20.941854,28.977273
Porciento de clientes atendidos por el modulo 4,2.342555,17.647059,26.340405,26.386323,26.432242,34.748011


In [35]:
pd.DataFrame({
    **worker_1_time,
    **worker_2_time,
    **worker_3_time,
    **worker_4_time, 
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Porcentaje de tiempo de trabajo el modulo 1 empleado en la atención de clientes,4.884373,69.094649,94.281461,94.377204,94.472948,111.481161
Porcentaje de tiempo de trabajo el modulo 2 empleado en la atención de clientes,3.653784,76.476184,97.750334,97.821956,97.893577,112.153730
Porcentaje de tiempo de trabajo el modulo 3 empleado en la atención de clientes,8.382979,49.117568,81.833822,81.998145,82.162469,107.376635
Porcentaje de tiempo de trabajo el modulo 4 empleado en la atención de clientes,6.864045,59.026616,90.162441,90.296990,90.431539,109.531499
